### Курсовой проект

построение модели
для прогнозирования невыполнения долговых обязательств по текущему кредиту

#### Содержание проекта:

**Построение модели классификации**<br>
[1. Обзор обучающего датасета](#1)<br>
[2. Обработка выбросов](#2)<br>
[3. Обработка пропусков](#3)<br>
[4. Анализ данных](#4)<br>
[5. Отбор признаков](#5)<br>
[6. Балансировка классов](#6)<br>
[7. Подбор моделей, получение бейзлана](#7)<br>
[8. Выбор наилучшей модели, настройка гиперпараметров](#8)<br>
[9. Проверка качества, борьба с переобучением](#9)<br>
[10. Интерпретация результатов](#10)<br>

**Прогнозирование на тестовом датасете**<br>
[11. Выполнить для тестового датасета те же этапы обработки и постронияния признаков](#11)<br>
[12. Спрогнозировать целевую переменную, используя модель, построенную на обучающем датасете](#12)<br>

#### Практическая часть

In [1]:
# пути к директориям и файлам
TRAIN_DATASET_PATH = 'data/course_project_train.csv'
TEST_DATASET_PATH = 'data/course_project_test.csv'

In [2]:
# импорт необходимых библиотек
import numpy as np
import pandas as pd
import pickle
from pathlib import Path

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split, ShuffleSplit, cross_val_score, learning_curve
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score
from sklearn.model_selection import StratifiedKFold, GridSearchCV, RandomizedSearchCV

from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.svm import SVC
import xgboost as xgb
import catboost as catb

import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')



ModuleNotFoundError: No module named 'xgboost'

##### Построение модели классификации

In [ ]:
# custom methods

# функция для просмотра номинативных признаков
def catnames_overview(data):
    for cat_colname in data.select_dtypes(include='object').columns:
        print(str(cat_colname) + '\n\n' + str(data[cat_colname].value_counts()) + '\n' + '*' * 100 + '\n')

In [ ]:
# метод для заполнения пропусков

def imputer_rfr(data, target_col):
    data = data.copy()
    
    features = data.columns
    
    data = data[features]
    
    train = data[~data[target_col].isna()]
    predict_data = data[data[target_col].isna()]

    X = train.drop(columns=target_col)
    y = train[target_col]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                        test_size=0.2,
                                                        shuffle=True,
                                                        random_state=32)
    
    model = RandomForestRegressor(n_estimators=100,
                                  max_depth=10,
                                  random_state=42,
                                  verbose=1)
    model.fit(X_train, y_train)
    
    pred_train = model.predict(X_train)
    pred_test = model.predict(X_test)
    
    print(f"r2 на train: {r2_score(y_train, pred_train)}")
    print(f"r2 на test: {r2_score(y_test, pred_test)}")

    pred = model.predict(predict_data.drop(columns=target_col))

    data.loc[data[target_col].isna(), target_col] = list(pred)
    return model, data

##### 1. Обзор обучающего датасета <a class="anchor" id="1"></a>

**Описание датасета**

* **Home Ownership** - домовладение
* **Annual Income** - годовой доход
* **Years in current job** - количество лет на текущем месте работы
* **Tax Liens** - налоговые обременения
* **Number of Open Accounts** - количество открытых счетов
* **Years of Credit History** - количество лет кредитной истории
* **Maximum Open Credit** - наибольший открытый кредит
* **Number of Credit Problems** - количество проблем с кредитом
* **Months since last delinquent** - количество месяцев с последней просрочки платежа
* **Bankruptcies** - банкротства
* **Purpose** - цель кредита
* **Term** - срок кредита
* **Current Loan Amount** - текущая сумма кредита
* **Current Credit Balance** - текущий кредитный баланс
* **Monthly Debt** - ежемесячный долг
* **Credit Default** - факт невыполнения кредитных обязательств (0 - погашен вовремя, 1 - просрочка)

In [ ]:
# загрузим данные
df_train = pd.read_csv(TRAIN_DATASET_PATH)
df_test = pd.read_csv(TEST_DATASET_PATH)

In [ ]:
# посмотрим какие данные есть в колонках
df_train.head()

In [ ]:
df_train.iloc[0]

In [ ]:
# размерность нашего набора данных
df_train.shape

In [ ]:
# посмотрим есть ли пропуски
# и какие типы данных представлены
df_train.info()

In [ ]:
# из списка видим, что есть пропуски
# и поэтому понадобиться их заполнить

In [ ]:
# Обзор целевой переменной
df_train['Credit Default'].value_counts()

In [ ]:
# Обзор количественных (вещественных) признаков
# сравним, если mean > 50% 
# это указывает на наличие выбросов в большую сторону
df_train.describe()

In [ ]:
# Обзор категориальных (номинативных) признаков
catnames_overview(df_train)

In [ ]:
# рассмотрим где есть пропущенные данные
df_train.isna().sum()

In [ ]:
# Выделение признакового описания и целевой переменной

TARGET_NAME = 'Credit Default'

BASE_FEATURE_NAMES = ['Home Ownership', 'Annual Income', 'Years in current job',
                     'Tax Liens', 'Number of Open Accounts', 'Years of Credit History',
                     'Maximum Open Credit', 'Number of Credit Problems', 
                     'Months since last delinquent',
                     'Bankruptcies', 'Purpose', 'Term', 'Current Loan Amount',
                     'Current Credit Balance', 'Monthly Debt', 'Credit Score']

CAT_FEATURE_NAMES = ['Home Ownership', 'Years in current job', 'Purpose', 'Term']

NUM_FEATURE_NAMES = ['Annual Income', 'Tax Liens', 
                     'Number of Open Accounts', 'Years of Credit History',
                     'Maximum Open Credit', 'Number of Credit Problems', 
                     'Months since last delinquent',
                     'Bankruptcies', 'Current Loan Amount',
                     'Current Credit Balance', 'Monthly Debt', 'Credit Score']

##### 2. Обработка выбросов <a class="anchor" id="2"></a>

In [ ]:
df_train[NUM_FEATURE_NAMES].hist(figsize=(16,16), bins=20, grid=False);

##### 3. Обработка пропусков <a class="anchor" id="3"></a>

In [ ]:
df_train_copy = df_train.copy()

In [ ]:
feature_name = 'Annual Income'
annual_income_predictor, df_train_copy = imputer_rfr(df_train_copy, feature_name)

##### 4. Анализ данных <a class="anchor" id="4"></a>

##### 5. Отбор признаков <a class="anchor" id="5"></a>

##### 6. Балансировка классов <a class="anchor" id="6"></a>

##### 7. Подбор моделей, получение бейзлана <a class="anchor" id="7"></a>

##### 8. Выбор наилучшей модели, настройка гиперпараметров <a class="anchor" id="8"></a>

##### 9. Проверка качества, борьба с переобучением <a class="anchor" id="9"></a>

##### 10. Интерпретация результатов <a class="anchor" id="10"></a>

##### Прогнозирование на тестовом датасете

##### 11. Выполнить для тестового датасета те же этапы обработки и построения признаков <a class="anchor" id="11"></a>

In [ ]:
# посмотрим какие данные есть в колонках
df_test.head()

In [ ]:
df_test.iloc[0]

In [ ]:
# размерность нашего набора данных
df_test.shape

In [ ]:
# посмотрим есть ли пропуски
# и какие типы данных представлены
df_test.info()

In [ ]:
# из списка видим, что есть пропуски
# и поэтому понадобиться их заполнить

In [ ]:
# Обзор количественных (вещественных) признаков
# сравним, если mean > 50% 
# это указывает на наличие выбросов в большую сторону
df_test.describe()

##### 12. Спрогнозировать целевую переменную, используя модель, построенную на обучающем датасете <a class="anchor" id="12"></a>